In [37]:
from google.colab import drive
drive.mount('/content/drive')
import cv2
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import ImageGrid
%matplotlib inline
import math




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
import os
import pandas as pd

dataset = []

for i in range(50):
  dataset.append([0,f'/content/drive/MyDrive/pro/img/train/dog/dog.{i}.jpg'])
for i in range(50):
  dataset.append([1,f'/content/drive/MyDrive/pro/img/train/cat/cat.{i}.jpg'])


In [ ]:
import cv2

import numpy as np

# 辞書サイズ(2なのは犬・猫の2種類に分類するモデルであるため)

dictionarySize = 2

# KAZE特徴量抽出器

detector = cv2.KAZE_create()
bf = cv2.BFMatcher(cv2.NORM_L2,False)
# Bag Of Visual Words分類器

bowTrainer = cv2.BOWKMeansTrainer(dictionarySize)

# 各画像を分析

for i, (classId, data_path) in enumerate(dataset):
    # 進捗表示

    # print(classId, data_path)

    # グレースケールで画像読み込み

    gray = cv2.imread(data_path, 1)

    gray = cv2.resize(gray, dsize=(300, 300))

    # 特徴点とその特徴を計算

    keypoints, descriptors= detector.detectAndCompute(gray, None)

    # intからfloat32に変換
    if descriptors is not None:
        descriptors = descriptors.astype(np.float32)
        # 特徴ベクトルをBag Of Visual Words分類器にセット

        bowTrainer.add(descriptors)


# Bag Of Visual Words分類器で特徴ベクトルを分類

codebook = bowTrainer.cluster()

# 訓練完了

print("train finish")


In [ ]:
result_index=[]
def kaze_train(img):
  matcher = cv2.BFMatcher()

  # Bag Of Visual Words抽出器
  bowExtractor = cv2.BOWImgDescriptorExtractor(detector, matcher)
  # トレーニング結果をセット
  bowExtractor.setVocabulary(codebook)
  # グレースケールで読み込み
  gray = cv2.imread(img,cv2.IMREAD_GRAYSCALE)

  # 画像をリサイズする
  gray = cv2.resize(gray, dsize=(300, 300))

  # 特徴点と特徴ベクトルを計算
  keypoints, descriptors= detector.detectAndCompute(gray, None)
  if descriptors is not None:
      # intからfloat32に変換
      descriptors = descriptors.astype(np.float32)
      # Bag Of Visual Wordsの計算
      bowDescriptors = bowExtractor.compute(gray, keypoints)
      className = {"0": "犬", "1": "猫"}
      actual = "???"
    # 分類確率が最大のインデックスを特定
      max_index = np.argmax(bowDescriptors[0])
      print(className[f"{max_index}"])
      result_index.append(max_index)
  else:
      print('fail')

In [ ]:

for i in range(30):
  kaze_train(f'/content/drive/MyDrive/pro/img/test/test.{i}.jpg')
seikai=[]
fuseikai=[]

for i,j in enumerate(result_index):
  if i <15:
    if j==0:
      seikai.append(1)
    else:
      fuseikai.append(i)
  if i >=15:
    if j==1:
      seikai.append(1)
    else:
      fuseikai.append(i)

print(f"正解率：{len(seikai)/(len(seikai)+len(fuseikai))*100}")